## ArcGIS Notebook to be demonstrate the usage of the built-in [Spark](https://spark.apache.org/) engine.


This notebook will demonstrate the spatial binning of AIS data around the port of Miami using Apache Spark.

The AIS broadcast data is in a FileGeodatabase that can be download from [here](https://marinecadastre.gov/ais). Note only data from 2010-2014 is in FileGeodatabase format.

### Import the required modules.

In [1]:
import os
import arcpy

from spark_esri import spark_start, spark_stop

### Start a Spark instance.

Note the `config` argument to [configure the Spark instance](https://spark.apache.org/docs/latest/configuration.html).

In [2]:
config = {
    "spark.driver.memory":"16G",
    "spark.executor.memory":"16G"
}

spark_stop()
spark = spark_start(config=config)

### Let's check the spark version and some env vars.

In [3]:
spark.version

'3.3.0-esri-6'

In [4]:
"CONDA_DEFAULT_ENV" in os.environ # Should be False, this was for Pro pre 2.8

False

In [5]:
os.getenv("CONDA_DEFAULT_ENV") # if set, what is the value ? Spark will use the python.exe in that folder.

In [6]:
os.environ["JAVA_HOME"]

'c:\\program files\\arcgis\\pro\\Java\\runtime\\jre'

In [7]:
os.environ["HADOOP_HOME"]

'C:\\Program Files\\ArcGIS\\Pro\\java\\runtime\\hadoop'

In [8]:
os.environ["SPARK_HOME"]

'C:\\Program Files\\ArcGIS\\Pro\\java\\runtime\\spark'

In [9]:
os.environ["PYSPARK_PYTHON"]

'C:\\Users\\mraad\\AppData\\Local\\ESRI\\conda\\envs\\sparkgeo\\python.exe'

### Read the selected Broadcast feature shapes in WebMercator SR.

It is assumed that you added to the map the `Broadcast` point feature class from the download `miami.gdb`.

Note that the `SearchCursor` is subject to the user selected features, and to an active query definition in the layer properties. For Example, set the query definition to `Stauts = 0: Under way using engine` to get the location of all moving ships, in such that we get a "heat map" of the port movement.

In [ ]:
sp_ref = arcpy.SpatialReference(3857)
data = arcpy.da.SearchCursor("Broadcast", ["SHAPE@X","SHAPE@Y"], spatial_reference=sp_ref)

### Create a Spark data frame of the read data, and create a view named 'v0'.

In [ ]:
spark\
    .createDataFrame(data, "x double,y double")\
    .createOrReplaceTempView("v0")

### Aggregate the data at 200x200 meters bins.

The aggregation is performed by Spark as a SQL statement in a parallel share-nothing way and the resulting bins are collected back in the `rows` array variable.

This is a nested SQL expression, where the inner expression is mapping the input `x` and `y` into `q` and `r` cell locations given a user defined bin size, and the outter expression is aggreating as a sum the `q` and `r` pairs. Finally, `q` and `r` are mapped back to `x` and `y` to enble the placement on a map. 

In [ ]:
cell0 = 200.0 # meters
cell1 = cell0 * 0.5

rows = spark\
    .sql(f"""
with t as (select cast(x/{cell0} as long) q,cast(y/{cell0} as long) r from v0)

select q*{cell0}+{cell1} x,r*{cell0}+{cell1} y,least(count(1),400) as pop
from t
group by q,r
""")\
    .collect()

### Report the number of bins

In [ ]:
len(rows)

### Create an in-memory point feature class of the collected bins.

The variable `rows` is an array of form `[[x0,y0,pop0],[x1,y1,pop1],...,[xN,yN,popN]]`.

In [ ]:
ws = "memory"
nm = "Bins"

fc = os.path.join(ws,nm)

arcpy.management.Delete(fc)

sp_ref = arcpy.SpatialReference(3857)
arcpy.management.CreateFeatureclass(ws,nm,"POINT",spatial_reference=sp_ref)
arcpy.management.AddField(fc, "POP", "LONG")

with arcpy.da.InsertCursor(fc, ["SHAPE@X","SHAPE@Y", "POP"]) as cursor:
    for row in rows:
        cursor.insertRow(row)

### Apply a graduated colors symbology to highlight the bins.

In [ ]:
# _ = arcpy.ApplySymbologyFromLayer_management(fc, f"{nm}.lyrx")

### Stop the spark instance.

In [ ]:
spark_stop()